In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from transformers import ViTImageProcessor, ViTForImageClassification
from PIL import Image
import requests
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import numpy as np

In [2]:
processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224')
model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')


In [3]:
# Define a custom dataset class
class CustomDataset(Dataset):
    def __init__(self, data, labels, transform=None):
        self.data = data
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        label = self.labels[idx]

        if self.transform:
            sample = self.transform(sample)

        return sample, label

In [4]:
from torchvision.datasets import ImageFolder
from torchvision.transforms import transforms

# Define data transforms (you can modify these based on your needs)
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to a consistent size
    transforms.ToTensor(),           # Convert images to tensors
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),  # Normalize pixel values
])

# Define paths to your data folders
train_data_dir = '/u/45/muhammu2/data/Desktop/sample/train/'
val_data_dir = '/u/45/muhammu2/data/Desktop/sample/validation/'
test_data_dir = '/u/45/muhammu2/data/Desktop/sample/test/'

# Load your training, validation, and test datasets using ImageFolder
train_dataset = ImageFolder(root=train_data_dir, transform=transform)
val_dataset = ImageFolder(root=val_data_dir, transform=transform)
test_dataset = ImageFolder(root=test_data_dir, transform=transform)

# If you want to access the labels for train, validation, and test datasets:
train_labels = train_dataset.targets
val_labels = val_dataset.targets
test_labels = test_dataset.targets

# Create data loaders
batch_size = 8  # Adjust the batch size as needed
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [5]:
# Define a binary classification head
class BinaryClassificationHead(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(BinaryClassificationHead, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Modify the Swin Transformer model for binary classification
classifier_head = BinaryClassificationHead(768, 32)  # Adjust input size as needed
model.classifier = classifier_head

# Define loss function and optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-4)


In [6]:
# Training loop
num_epochs = 10  # Modify this as needed
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for data, labels in train_loader:
        data, labels = data.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(data).logits
        loss = criterion(outputs, labels.unsqueeze(1).float())
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")


Epoch 1, Loss: 0.2675505539816287
Epoch 2, Loss: 0.05120344125251803
Epoch 3, Loss: 0.049050706945773626
Epoch 4, Loss: 0.037369880049178995
Epoch 5, Loss: 0.003922998147188789
Epoch 6, Loss: 0.002384590680917932
Epoch 7, Loss: 0.0017179004007226062
Epoch 8, Loss: 0.0013130273062011435
Epoch 9, Loss: 0.001040469294336314
Epoch 10, Loss: 0.0008459321449562493


In [7]:
# Validation set for EER calculation
model.eval()
val_labels, val_scores = [], []

with torch.no_grad():
    for data, labels in val_loader:
        data, labels = data.to(device), labels.to(device)
        outputs = model(data).logits
        val_labels.extend(labels.cpu().numpy())
        val_scores.extend(torch.sigmoid(outputs).cpu().numpy())


# Calculate Equal Error Rate (EER) on the validation set
fpr, tpr, thresholds = roc_curve(val_labels, val_scores, pos_label=1)
eer_threshold = thresholds[np.argmin(np.abs(tpr - (1 - fpr)))]
eer = 1 - tpr[np.argmin(np.abs(tpr - (1 - fpr)))]
print(f'Validation EER: {eer*100:.4f}, EER Threshold: {eer_threshold:.4f}')

Validation EER: 0.0000, EER Threshold: 0.0038


In [8]:

# Initialize lists to store test scores and labels
test_labels, test_scores = [], []

# Make predictions on the test set
model.eval()
with torch.no_grad():
    for data, labels in test_loader:
        data, labels = data.to(device), labels.to(device)
        outputs = model(data).logits
        test_labels.extend(labels.cpu().numpy())
        test_scores.extend(torch.sigmoid(outputs).cpu().numpy())

# Calculate the HTER on the testing set using the EER threshold
threshold = eer_threshold
predicted_labels = [1 if score > threshold else 0 for score in test_scores]

false_acceptance = sum(1 for i in range(len(predicted_labels)) if predicted_labels[i] == 1 and test_labels[i] == 0)
false_rejection = sum(1 for i in range(len(predicted_labels)) if predicted_labels[i] == 0 and test_labels[i] == 1)

total_samples = len(test_labels)
hter = ((false_acceptance + false_rejection) / (2 * total_samples)) * 100
print(f"HTER using EER threshold: {hter:.2f}%")


HTER using EER threshold: 34.86%


In [9]:
# Calculate the AUC on the testing set
from sklearn.metrics import roc_auc_score
auc_score = roc_auc_score(test_labels, test_scores)
print(f"AUC on the testing set: {auc_score*100:.2f}")

AUC on the testing set: 68.13
